In [ ]:
import cv2
from keras.models import load_model
import numpy as np
from pygame import mixer
import time
from lenet import LeNet
from keras.preprocessing.image import img_to_array
import imutils
import statistics   

mixer.init()
sound = mixer.Sound('alarm.wav')
face = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
lbl=['Sleepy','NoSleepy']
model = LeNet.build(width=28, height=28, depth=1, classes=2)
model.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])
model.load_weights('my_model_weights_final_friends_all.h5')
#model.load_weights('transfer_learning.h5')
#cap = cv2.VideoCapture('./Fold3_part1/17/0.mp4')
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
thicc=2
framNum=0
allprop=[]
Rotate='UP'
while(True):
    ret, frame = cap.read()
    ret, frame = cap.read()
    if (Rotate=='Down'):
            frame=imutils.rotate_bound(frame,angle=180)

    if (Rotate=='Left'):
            frame=imutils.rotate_bound(frame,angle=90)

    if (Rotate=='Right'):
            frame=imutils.rotate_bound(frame,angle=270)

    height,width = frame.shape[:2]           
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    start_time = time.time()
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )
        data=frame[y:y+h,x:x+w]
        data = cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)
        data = cv2.resize(data, (28, 28), interpolation = cv2.INTER_AREA)
        #data=cv2.equalizeHist(data)
        datalist=[]
        datalist.append(img_to_array(data))      
        data=np.array(datalist,dtype=float)/255.0
        prediction = model.predict(data,batch_size=100)
        print("--- %s seconds ---" % (time.time() - start_time))
        allprop.append(prediction[0,0]*100)
        cv2.putText(frame,'Drowsiness score:   '+str(statistics.mean(allprop)),(20,height-20), font, 0.75,(255,255,255),1,cv2.LINE_AA)
        print('Drowsiness Score:'+str(statistics.mean(allprop)))
        if(statistics.mean(allprop)>40):
            try:
                sound.play()        
            except: 
                pass
        if(thicc<10):
            thicc= thicc+2
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc) 
        cv2.imshow('Drowsiness Detection',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()